In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/"))





path,name,size,modificationTime
dbfs:/FileStore/ipl_cleaned.parquet/,ipl_cleaned.parquet/,0,0
dbfs:/FileStore/ipl_final_cleaned.parquet/,ipl_final_cleaned.parquet/,0,0
dbfs:/FileStore/ipl_validated.parquet/,ipl_validated.parquet/,0,0
dbfs:/FileStore/tables/,tables/,0,0


In [0]:

df = spark.read.format("parquet").load("dbfs:/FileStore/ipl_final_cleaned.parquet/")

df.show(5, truncate=False)


+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|year|series_type        |series_name               |match_no|match_type|match_name                                              |match_href                                                                                   |match_team1          |match_team2                |match_datetime_start|match_date_end|match_venue|
+----+-------------------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------+---------------------------+--------------------+--------------+-----------+
|2021|2025-04-09 20:00:00|India

In [0]:
from pyspark.sql import Row

commentary_rows = [
    Row(over=1.1, ball_type="normal", commentary="Player A hits a four", team="Team A"),
    Row(over=1.2, ball_type="wide", commentary="Wide ball by Player B", team="Team B")
]

commentary_df = spark.createDataFrame(commentary_rows)
commentary_df.show()


+----+---------+--------------------+------+
|over|ball_type|          commentary|  team|
+----+---------+--------------------+------+
| 1.1|   normal|Player A hits a four|Team A|
| 1.2|     wide|Wide ball by Play...|Team B|
+----+---------+--------------------+------+



In [0]:

over_rows = [
    Row(over=1.1, ball_type="normal", commentary="No run", team="Team A"),
    Row(over=1.2, ball_type="no-ball", commentary="No ball and run", team="Team B")
]

over_df = spark.createDataFrame(over_rows)
over_df.show()


+----+---------+---------------+------+
|over|ball_type|     commentary|  team|
+----+---------+---------------+------+
| 1.1|   normal|         No run|Team A|
| 1.2|  no-ball|No ball and run|Team B|
+----+---------+---------------+------+



In [0]:

combined_union = commentary_df.union(over_df)
print("Using union():")
combined_union.show()


Using union():
+----+---------+--------------------+------+
|over|ball_type|          commentary|  team|
+----+---------+--------------------+------+
| 1.1|   normal|Player A hits a four|Team A|
| 1.2|     wide|Wide ball by Play...|Team B|
| 1.1|   normal|              No run|Team A|
| 1.2|  no-ball|     No ball and run|Team B|
+----+---------+--------------------+------+



In [0]:

combined_union_by_name = commentary_df.unionByName(over_df)
print("Using unionByName():")
combined_union_by_name.show()


Using unionByName():
+----+---------+--------------------+------+
|over|ball_type|          commentary|  team|
+----+---------+--------------------+------+
| 1.1|   normal|Player A hits a four|Team A|
| 1.2|     wide|Wide ball by Play...|Team B|
| 1.1|   normal|              No run|Team A|
| 1.2|  no-ball|     No ball and run|Team B|
+----+---------+--------------------+------+



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2003377816745623>:2
      1 # Load commentary data
----> 2 commentary_df = spark.read.parquet("dbfs:/FileStore/tables/commentary_data.parquet")
      4 # Load over data
      5 over_df = spark.read.parquet("dbfs:/FileStore/tables/over_data.parquet")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:533, in DataFrameReader.parquet(self, *paths, **options)
    522 int96RebaseMode = options.get("int96RebaseMode", None)
    523 self._set_opts(
    524   